## 1. Why decorators exist

In Python, **functions are first-class objects**:


When we say **“functions are first-class objects”**, it means:

> **Functions are treated like any other value in the language**
> (just like integers, strings, or objects)

In Python, this has very concrete and practical consequences.

---

#### 1.1. What “first-class” means (precisely)

An entity is *first-class* if it can:

- ✅ be assigned to a variable
- ✅ be passed as an argument to a function
- ✅ be returned from a function
- ✅ be stored in data structures

Python functions satisfy **all four**.

---

#### 1.2. Functions can be assigned to variables

```python
def greet():
    print("Hello")

f = greet      # no parentheses
f()
```

Key point:

* `greet` is the **function object**
* `greet()` is the **result of calling it**

---

#### 1.3. Functions can be passed as arguments

```python
def run(func):
    func()

def say_hi():
    print("Hi")

run(say_hi)
```

This works because `say_hi` is just a value.

---

#### 1.4. Functions can be returned from other functions

```python
def make_adder(x):
    def add(y):
        return x + y
    return add

add5 = make_adder(5)
print(add5(3))
```

Output:

```
8
```

Here:

* `add` is created dynamically
* returned as a value
* stored in `add5`

---

#### 1.5. Functions can be stored in data structures

```python
def add(a, b):
    return a + b

def sub(a, b):
    return a - b

ops = {
    "add": add,
    "sub": sub
}

print(ops["add"](2, 1))
```

---

#### 1.6. Functions have identity and attributes

Functions are objects:

```python
def f():
    pass

print(type(f))
print(f.__name__)
print(f.__doc__)
```

Output:

```
<class 'function'>
f
None
```

They live in memory and have metadata.

---

#### 1.7. Why this matters (the big picture)

Because functions are first-class:

* decorators are possible
* callbacks are possible
* event systems work
* web frameworks (FastAPI, Flask) work
* functional programming patterns work

Example (decorator connection):

```python
def decorator(func):
    def wrapper():
        print("Before")
        func()
        print("After")
    return wrapper
```

This only works because `func` is a value.

---

#### 1.8. Contrast with languages where functions are NOT first-class

In older languages (or limited ones):

❌ functions cannot be passed as values
❌ callbacks require special syntax
❌ decorators are impossible

Python does not have these limitations.

---

#### 1.9. Very short mental model

Think of a function as:

> “A value that happens to be callable”

Just like:

```python
x = 5
x()

❌ TypeError
```

but:

```python
f = greet
f()

✅
```

---



A **decorator** lets you:

* Wrap a function
* Add behavior **before and/or after** it
* Without modifying the original function code

Typical uses:

* Logging
* Timing
* Authentication / authorization
* Caching
* Input validation

---

## 2. Functions as objects

```python
def greet():
    print("Hello")

f = greet
f()
```

Output:

```
Hello
```

You assigned a function to another name and called it.

---

## 3. Function inside a function

```python
def outer():
    def inner():
        print("Inner function")
    inner()

outer()
```

This shows that functions can be **defined and called inside other functions**.

---

## 4. Returning a function

```python
def make_printer():
    def printer():
        print("Printing...")
    return printer

p = make_printer()
p()
```

Here, `make_printer` returns a function.

---

## 5. A simple decorator (manual version)

Goal: add behavior around a function.

```python
def my_decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper
```

Use it manually:

```python
def say_hello():
    print("Hello!")

say_hello = my_decorator(say_hello)
say_hello()
```

Output:

```
Before function call
Hello!
After function call
```

---

## 6. The `@decorator` syntax (clean way)

Python provides syntax sugar:

```python
def my_decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

This is **exactly the same** as:

```python
say_hello = my_decorator(say_hello)
```

---

## 7. Decorators with arguments to the function

If the decorated function takes arguments:

```python
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before")
        result = func(*args, **kwargs)
        print("After")
        return result
    return wrapper
```

Example:

```python
@my_decorator
def add(a, b):
    return a + b

print(add(2, 3))
```

Output:

```
Before
After
5
```

---

## 8. Preserving function metadata (`functools.wraps`)

Without this, the function name and docstring are lost.

```python
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper
```

Now:

```python
print(add.__name__)
```

Correctly prints:

```
add
```

---

## 9. Decorators with parameters (advanced but common)

A decorator that **itself takes arguments**:

```python
def repeat(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator
```

Usage:

```python
@repeat(3)
def hello():
    print("Hello")

hello()
```

Output:

```
Hello
Hello
Hello
```

---

## 10. Real-world example: timing a function

```python
import time
from functools import wraps

def timing(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Execution time: {end - start:.4f}s")
        return result
    return wrapper
```

Usage:

```python
@timing
def slow_function():
    time.sleep(1)

slow_function()
```

---

## 11. Mental model (important)

When you see:

```python
@decorator
def f():
    pass
```

Think:

```python
f = decorator(f)
```

That’s it. No magic.

---

## 12. FastAPI Decorators


```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/ping")
def ping():
    return {"message": "pong"}
```

---

## 2. What `@app.get("/ping")` actually means

This line:

```python
@app.get("/ping")
```

means:

> Call the method `get` on the `app` object, passing `"/ping"`
> The result must be a **decorator**

So internally this is equivalent to:

```python
decorator = app.get("/ping")
ping = decorator(ping)
```

This is **pure Python decorator mechanics**.

---

## 3. What does `app.get("/ping")` return?

It returns a **decorator function**.

Conceptually:

```python
def get(self, path):
    def decorator(func):
        # register the function as a route
        self.routes.append({
            "path": path,
            "method": "GET",
            "handler": func
        })
        return func
    return decorator
```

So when Python sees:

```python
@app.get("/ping")
def ping():
    ...
```

It becomes:

```python
ping = app.get("/ping")(ping)
```

---

## 4. Key difference from your earlier decorator examples

Earlier decorators:

```python
@my_decorator
def f():
    ...
```

* **Wrap the function**
* Replace it with another function (`wrapper`)
* Change runtime behavior when called

FastAPI decorators:

```python
@app.get("/ping")
def ping():
    ...
```

* **Do NOT wrap the function**
* Do NOT change how `ping()` behaves
* They **register metadata** about the function

So:

```python
ping()
```

still works normally.

---

## 5. What FastAPI does with the function

When Python loads the file:

1. The function `ping` is defined
2. `app.get("/ping")` is executed
3. The function is **registered as a route**
4. The function is returned unchanged

Later, when an HTTP request comes in:

```
GET /ping
```

FastAPI:

* finds the route `/ping`
* sees it maps to `ping`
* calls `ping()`
* serializes the return value to JSON

---

## 6. Why FastAPI uses decorators instead of function calls

Compare these two styles.

### Without decorators (ugly)

```python
def ping():
    return {"message": "pong"}

app.add_route("/ping", ping, methods=["GET"])
```

### With decorators (clean)

```python
@app.get("/ping")
def ping():
    return {"message": "pong"}
```

Decorators allow:

* route definition **next to the function**
* better readability
* less boilerplate
* static analysis (type hints, OpenAPI generation)

---

## 7. Decorator stacking in FastAPI

You can stack decorators because they are just functions:

```python
@app.get("/ping")
@timing
def ping():
    return {"message": "pong"}
```

Execution order:

1. `timing(ping)` runs first
2. `app.get("/ping")` registers the wrapped function

---

## 8. Why this feels “different”

Because:

| Normal decorator           | FastAPI decorator       |
| -------------------------- | ----------------------- |
| Wraps behavior             | Registers metadata      |
| Changes function execution | Leaves function intact  |
| Called when function runs  | Used during app startup |
| Runtime logic              | Configuration logic     |

FastAPI decorators are **configuration-time decorators**, not behavior decorators.

---

## 9. Mental model (very important)

When you see:

```python
@app.get("/ping")
def ping():
    ...
```

Think:

> “Register this function as a GET endpoint at `/ping`”

Not:

> “Modify this function’s behavior”

---

## 10. Why this is powerful

Because FastAPI can:

* auto-generate OpenAPI docs
* validate inputs using type hints
* inject dependencies
* serialize outputs
* handle async/sync automatically

All from the **function signature**.

---
